In [1]:
import os
import multiprocessing
import time
import numpy as np
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit, Parameter
from qiskit import Aer, execute
from numpy.random import default_rng
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
import numba

@numba.vectorize([numba.float64(numba.complex128), numba.float32(numba.complex64)], target='parallel')
def abs2(x):
    return x.real**2 + x.imag**2

In [3]:
def one_step(dt, m=0.5, w=1., J=1., barrier=False):
    num_sites = system_size
    
    qc = QuantumCircuit(num_sites)
    
    # exp(-i dt Hzz)
    for iqr in range(1, num_sites - 1):
        phi = J * 0.5 * (2. * dt) * (num_sites - iqr - 1)
        for iql in range(iqr):
            qc.cx(iql, iqr)
            qc.p(phi, iqr)
            qc.cx(iql, iqr)
            
    if barrier:
        qc.barrier()
            
    # exp(-i dt Hpm)
    # Hpm = w sum (sigma+ sigma- + h.c.) = w sum h
    # h = 1/2 XX + 1/2 YY
    phi = w * 0.5 * (2. * dt)
    for iq in range(num_sites - 1):
        # exp(-i dt w/2 XX)
        qc.cx(iq, iq + 1)
        qc.rx(phi, iq)
        qc.cx(iq, iq + 1)
        # exp(-i dt w/2 YY)
        qc.cz(iq, iq + 1)
        qc.cx(iq, iq + 1)
        qc.rx(phi, iq)
        qc.cx(iq, iq + 1)
        qc.cz(iq, iq + 1)
        
    if barrier:
        qc.barrier()

    # exp(-i dt Hz1)
    phi = m * 0.5 * (2. * dt)
    for iq in range(num_sites):
        if iq % 2 == 0:
            qc.p(-phi, iq)
        else:
            qc.p(phi, iq)
            
    if barrier:
        qc.barrier()

    # exp(-i dt Hz2)
    for iq in range(num_sites - 1):
        phi = -J * 0.5 * (num_sites // 2 - (iq + 1) // 2) * (2. * dt)
        qc.p(phi, iq)
            
    if barrier:
        qc.barrier()
            
    return qc

In [4]:
def get_renyi_entropy_swaptest(circuit, subsystem):
    nq = circuit.num_qubits
    circ = QuantumCircuit(nq * 2 + 1, 1)
    circ = circ.compose(circuit, qubits=list(range(nq)))
    circ = circ.compose(circuit, qubits=list(range(nq, nq * 2)))

    # cswap only on the subsystem
    circ.h(nq * 2)
    for iq in subsystem:
        circ.cswap(nq * 2, iq, nq + iq)
    circ.h(nq * 2)
    circ.measure(nq * 2, 0)
    
    backend = Aer.get_backend('qasm_simulator')
    backend.set_options(method='statevector_gpu')
    shots = 2 ** 19
    job = execute(circ, backend, shots=shots)
    result = job.result()
    print(result.data(0)['counts'])
    zero_counts = result.data(0)['counts']['0x0']
    
    return zero_counts / shots * 2. - 1.

In [5]:
def get_renyi_entropy(statevector, subsystem):
    #system_size = int(np.log2(statevector.shape[0]))
    subsystem_size = len(subsystem)
    
    start = time.time()
        
    # assuming subsystem is contiguous
    low = min(subsystem)
    high = system_size - max(subsystem) - 1
    statevector = statevector.reshape([2 ** high, 2 ** subsystem_size, 2 ** low])

    treshape = time.time() - start

    # p_i p_j |<psi_i | psi_j>|^2
    # statevector components are unnormalized -> square(abs(statevector)) = p_i ||psi_i>|^2
    #purity = np.sum(np.square(np.abs(np.einsum('ik,jk->ij', np.conjugate(statevector), statevector)))) # by environment indices
    #purity = np.sum(np.square(np.abs(np.einsum('ij,ik->jk', np.conjugate(statevector), statevector)))) # by subsystem indices
    #purity = np.sum(abs2(einsum('ik,jk->ij', np.conjugate(statevector), statevector))) # by environment indices
    start = time.time()
    conj = np.conjugate(statevector)
    tconj = time.time() - start
    print(statevector.shape)
    start = time.time()
    #prod = np.matmul(conj.T, statevector)
    prod = np.tensordot(conj, statevector, axes=([0, 2], [0, 2]))
    #raise RuntimeError(prod)
    tprod = time.time() - start
    conj = None
    start = time.time()
    prob = abs2(prod)
    tprob = time.time() - start
    prod = None
    start = time.time()
    purity = np.sum(prob)
    tpurity = time.time() - start
    start = time.time()
    entropy = -np.log2(purity)
    tentropy = time.time() - start
    print('reshape', treshape, 'conj', tconj, 'prod', tprod, 'prob', tprob, 'purity', tpurity, 'entropy', tentropy)
    #purity = np.sum(abs2(np.matmul(np.conjugate(statevector), statevector.T)))
    
    # einsum is not parallelized
#    conj = np.conjugate(statevector)
#    outerprod = np.tensordot(conj, statevector, axes=([1], [1]))
#    purity = np.sum(abs2(outerprod))
    
    return -np.log2(purity)

In [6]:
# try:
#     os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda/lib64'
# except KeyError:
#     os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

MAX_SIZE = 30

@tf.function
def compute_purity_tf(statevector_real, statevector_imag):
#     prod_real = tf.tensordot(statevector_real, statevector_real, axes=[[0, 2], [0, 2]])
#     prod_real += tf.tensordot(statevector_imag, statevector_imag, axes=[[0, 2], [0, 2]])
#     prod_imag = tf.tensordot(statevector_real, statevector_imag, axes=[[0, 2], [0, 2]])
#     prod_imag -= tf.tensordot(statevector_imag, statevector_real, axes=[[0, 2], [0, 2]])
    prod_real = tf.tensordot(statevector_real, statevector_real, axes=[[1], [1]])
    prod_real += tf.tensordot(statevector_imag, statevector_imag, axes=[[1], [1]])
    prod_imag = tf.tensordot(statevector_real, statevector_imag, axes=[[1], [1]])
    prod_imag -= tf.tensordot(statevector_imag, statevector_real, axes=[[1], [1]])
#     purity = tf.tensordot(prod_real, prod_real, axes=[[0, 1], [0, 1]])
#     purity += tf.tensordot(prod_imag, prod_imag, axes=[[0, 1], [0, 1]])
    purity = tf.math.reduce_sum(tf.math.square(prod_real))
    purity += tf.math.reduce_sum(tf.math.square(prod_imag))
    
    return purity

def get_renyi_entropy_tf(statevector, subsystem):
    subsystem_size = len(subsystem)

    if system_size <= MAX_SIZE:
        low = min(subsystem)
        high = system_size - max(subsystem) - 1

        slices = [np.s_[:]]
    elif max(subsystem) < MAX_SIZE:
        low = min(subsystem)
        high = MAX_SIZE - max(subsystem) - 1

        block_size = 2 ** MAX_SIZE
        slices = [np.s_[block_size * i:block_size * (i + 1)] for i in range(2 ** (system_size - MAX_SIZE))]
    else:
        raise RuntimeError('don\'t do this it\'s too much work')
        
    shape = (2 ** high, 2 ** subsystem_size, 2 ** low)
    tfshape = (2 ** subsystem_size, 2 ** (high + low))
        
    purity = 0.
    for sl in slices:
        start = time.time()
        statevector = np.reshape(statevector[sl], shape).transpose((1, 0, 2)) # another reshape will cause data copy
        real = tf.constant(statevector.real, shape=tfshape)
        imag = tf.constant(statevector.imag, shape=tfshape)
        purity += compute_purity_tf(real, imag).numpy()
    entropy = -np.log2(purity)

    return entropy

In [7]:
import numba.cuda as ncuda

THREADS_PER_BLOCK = 64
CHUNK_SIZE = THREADS_PER_BLOCK

@ncuda.jit
def compute_purity_numba(real, imag, purity):
    """
    Each thread block computes sub-matrices of prod_real and prod_imag.
    Following implementation from https://docs.nvidia.com/cuda/cuda-c-programming-guide and https://numba.readthedocs.io/en/stable/cuda/examples.html
    Difference wrt reference: Result is not a matrix but a squared sum of all matrix elements -> no need to allocate a result matrix
    """
    
    x, y = ncuda.grid(2)
    
    tx = ncuda.threadIdx.x
    ty = ncuda.threadIdx.y
    
    real_sub = ncuda.shared.array((CHUNK_SIZE, CHUNK_SIZE), real.typestr)
    imag_sub = ncuda.shared.array((CHUNK_SIZE, CHUNK_SIZE), imag.typestr)
    
    res_real = 0.
    res_imag = 0.
    
    for ichunk in range(real.shape[1] // CHUNK_SIZE):
        real_sub[tx, ty] = real[x, ichunk * CHUNK_SIZE + ty]
        imag_sub[tx, ty] = imag[x, ichunk * CHUNK_SIZE + ty]
        
        ncuda.syncthreads()
        
        for ielem in range(CHUNK_SIZE):
            res_real += real_sub[tx, ielem] * real_sub[ty, ielem]
            res_real += imag_sub[tx, ielem] * imag_sub[ty, ielem]
            res_imag += real_sub[tx, ielem] * imag_sub[ty, ielem]
            res_imag -= imag_sub[tx, ielem] * real_sub[ty, ielem]
            
        ncuda.syncthreads()
        
    pos = ncuda.grid(1)
    purity[pos] = res_real * res_real + res_imag * res_imag

def get_renyi_entropy_numba(statevector, subsystem):
    subsystem_size = len(subsystem)

    if system_size <= MAX_SIZE:
        low = min(subsystem)
        high = system_size - max(subsystem) - 1

        slices = [np.s_[:]]
    elif max(subsystem) < MAX_SIZE:
        low = min(subsystem)
        high = MAX_SIZE - max(subsystem) - 1

        block_size = 2 ** MAX_SIZE
        slices = [np.s_[block_size * i:block_size * (i + 1)] for i in range(2 ** (system_size - MAX_SIZE))]
    else:
        raise RuntimeError('don\'t do this it\'s too much work')
        
    shape = (2 ** high, 2 ** subsystem_size, 2 ** low)
    devshape = (2 ** subsystem_size, 2 ** (high + low))
        
    purity = ncuda.device_array((subsystem_size, subsystem_size), dtype=np.float64)
    for sl in slices:
        start = time.time()
        statevector = np.reshape(statevector[sl], shape).transpose((1, 0, 2)) # another reshape will cause data copy

        real = ncuda.device_array(devshape, dtype=np.float64)
        imag = ncuda.device_array(devshape, dtype=np.float64)
        numba.cuda.to_device(statevector.real, to=real)
        numba.cuda.to_device(statevector.imag, to=imag)

        compute_purity_tf(real, imag, purity)
    
    entropy = -np.log2(np.sum(purity.copy_to_host()))

    return entropy   

In [8]:
def get_subsystem_probs(circuit):
#     backend = Aer.get_backend('statevector_simulator')
#     job = execute(circuit, backend)
#     result = job.result()
#     probs = np.square(np.abs(result.data()['statevector']))
    
    backend = Aer.get_backend('qasm_simulator')
    backend.set_options(method='statevector_gpu')
    circuit.measure_all()
    shots = 2 ** 16
    job = execute(circuit, backend, shots=2**16)
    result = job.result()
    
    probs = np.zeros(2 ** system_size, dtype=np.float)
    for hexidx, count in result.data(0)['counts'].items():
        probs[int(hexidx[:(system_size // 4) + 2], 16)] = count
    probs /= shots
        
    # take the subsystem from the center of the bitstring
    margin = 2 ** ((system_size - subsystem_size) // 2)
    probs = np.sum(probs.reshape((margin, 2 ** subsystem_size, margin)), axis=(0, 2))
    
    return probs

In [9]:
def get_electron_number_distribution(probs):
    result = np.zeros(subsystem_size + 1, dtype=np.float)
    for idx, prob in enumerate(probs):
        # up to subsystem_size == 8
        bitarray = np.unpackbits(np.array(idx, dtype=np.uint8))[-1:-subsystem_size - 1:-1]
        z = np.sum(1 - bitarray[0::2], dtype=np.int) - np.sum(bitarray[1::2], dtype=np.int) + (subsystem_size // 2)
        result[z] += prob
        
    return result

In [10]:
system_size = 30
subsystem = list(range(11, 19))
subsystem_size = len(subsystem)

m = 0.5
w = 1.
J = 1.

In [11]:
dt = np.pi / 16.
num_steps = 2

In [12]:
entropies = np.empty(num_steps + 1, dtype=np.float)
entropies[0] = 0.

circuit = QuantumCircuit(system_size)

# initial state = vacuum
for iq in range(0, system_size, 2):
    circuit.x(iq)

for istep in range(num_steps):
    start = time.time()
    circuit += one_step(dt)
    
    backend = Aer.get_backend('statevector_simulator')
    backend.set_options(method='statevector_gpu')
    job = execute(circuit, backend)
    result = job.result()
    statevector = result.data()['statevector']
    
    print(time.time() - start, 's to move one step')

    start = time.time()
    entropy = get_renyi_entropy(statevector, subsystem)
    entropies[istep + 1] = entropy
    print(time.time() - start, 's to compute the entropy')

In [13]:
for istep in range(num_steps):
    circuit += one_step(dt)
    if istep != num_steps - 1:
        circuit.snapshot('step{}'.format(istep))

backend = Aer.get_backend('statevector_simulator')
backend.set_options(method='statevector_gpu')

job = execute(circuit, backend)

data = job.result().data()

In [14]:
statevectors = [data['snapshots']['statevector']['step{}'.format(istep)][0] for istep in range(num_steps - 1)]
statevectors.append(data['statevector'])

for istep, statevector in enumerate(statevectors):
    start = time.time()
    entropy = get_renyi_entropy_tf(statevector, subsystem)
    entropies[istep + 1] = entropy
    print(time.time() - start, 's to compute the entropy')

ValueError: incompatible shape: (256, 4194304) vs. (256, 2048, 2048)

(4194304, 256)
conj 4.3823323249816895 prod 88.82510685920715 prob 0.00017189979553222656 purity 0.00012040138244628906
102.49019455909729 s to compute the entropy
(4194304, 256)
conj 4.1126720905303955 prod 11.836031675338745 prob 0.000141143798828125 purity 0.00011515617370605469
24.874769926071167 s to compute the entropy
(4194304, 256)
conj 4.122253656387329 prod 14.296188592910767 prob 0.00014519691467285156 purity 0.000118255615234375
27.454813718795776 s to compute the entropy
(4194304, 256)
conj 4.260644435882568 prod 13.633363485336304 prob 0.0001480579376220703 purity 0.0001227855682373047
27.13946557044983 s to compute the entropy
(4194304, 256)
conj 4.161118030548096 prod 15.46696400642395 prob 0.00014495849609375 purity 0.00011396408081054688
28.771482706069946 s to compute the entropy
(4194304, 256)
conj 4.131652116775513 prod 13.794524908065796 prob 0.0001430511474609375 purity 0.00011396408081054688
27.037447452545166 s to compute the entropy
(4194304, 256)
conj 4.1322996616363525 prod 13.098738431930542 prob 0.00014019012451171875 purity 0.00011086463928222656
26.27515149116516 s to compute the entropy
(4194304, 256)
conj 4.092943429946899 prod 12.78951621055603 prob 0.00014090538024902344 purity 0.00011277198791503906
25.930195808410645 s to compute the entropy
(4194304, 256)
conj 4.2148401737213135 prod 13.531028747558594 prob 0.00015115737915039062 purity 0.00011396408081054688
26.879091262817383 s to compute the entropy
(4194304, 256)
conj 4.336042881011963 prod 18.468639135360718 prob 0.00017786026000976562 purity 0.00011706352233886719
32.21967649459839 s to compute the entropy
plt.plot(np.arange(num_steps + 1), entropies / subsystem

(8192, 256, 512)
reshape 2.384185791015625e-05 conj 4.262945890426636 prod 89.08509635925293 prob 0.0003581047058105469 purity 0.0001227855682373047 entropy 2.0742416381835938e-05
93.38376212120056 s to compute the entropy
(8192, 256, 512)
reshape 1.2874603271484375e-05 conj 4.360467433929443 prod 26.1311252117157 prob 0.000156402587890625 purity 0.00011157989501953125 entropy 1.7404556274414062e-05
30.528235912322998 s to compute the entropy
(8192, 256, 512)
reshape 8.106231689453125e-06 conj 4.356942415237427 prod 26.82246732711792 prob 0.00014901161193847656 purity 0.00010704994201660156 entropy 1.7404556274414062e-05
31.217225313186646 s to compute the entropy

In [ ]:
plt.plot(np.arange(num_steps + 1), entropies / subsystem_size)

In [ ]:
low = min(subsystem)
high = system_size - max(subsystem) - 1
statevector = statevector.reshape([2 ** high, 2 ** subsystem_size, 2 ** low])

probs = np.sum(abs2(statevector), axis=(0, 2))
indices = np.arange(2 ** subsystem_size, dtype=np.uint8).reshape((-1, 1))
bitarray = np.unpackbits(indices, axis=1)
electron_numbers = np.sum(1 - bitarray[:, 0::2], axis=1, dtype=np.int) - np.sum(bitarray[:, 1::2], axis=1, dtype=np.int)

dist = np.zeros(subsystem_size + 1)
for n in range(-subsystem_size // 2, subsystem_size // 2 + 1):
    idx = np.nonzero(electron_numbers == n)[0]
    dist[n + subsystem_size // 2] += np.sum(probs[idx])

In [ ]:
plt.bar(np.linspace(-(subsystem_size // 2), subsystem_size // 2, subsystem_size + 1), dist)
plt.yscale('log')